In [1]:
pip install surprise

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 111.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=639542 sha256=6bf9a6ed384afdd298d3337c0f97e861809df5f3c90ac82871919464a21a02e0
  Stored in directory: /home/featurize/.cache/pip/wheels/99/90/55/3b18532fbb4958f22561f92df4507bbbb95cbe1cdaa490a13f
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
import numpy as np
import os

In [5]:
# Load the data
data_df = pd.read_csv('Ratings.csv')

In [6]:
data_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [21]:
# Load the jokes
books_df = pd.read_csv('Books.csv')

/tmp/ipykernel_20275/170401396.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('Books.csv')


In [22]:
#将books_df中的索引定义为Book-Title
books_df = books_df.rename_axis('book_id').reset_index()

In [23]:
#完成实验步骤中对数据集操作后执行
books_df.head()

,book_id,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [24]:
# Create a Reader object
reader = Reader(rating_scale=(0, 10))

In [26]:
ratings = data_df.sort_values('User-ID')

In [27]:
split_user_id =  ratings['User-ID'].quantile(0.8)

In [28]:
# 创建训练集和测试集
train_ratings = ratings[ratings['User-ID']<split_user_id]
test_ratings = ratings[ratings['User-ID']>=split_user_id]

In [30]:
# 将训练集和测试集分别转换为Surprise的格式
train_data = Dataset.load_from_df(train_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset = train_data.build_full_trainset()

In [31]:
testset = [tuple(x) for x in test_ratings[['User-ID', 'ISBN', 'Book-Rating']].values]

In [32]:
# 使用SVD算法
algo = SVD()

In [33]:
# 训练模型
algo.fit(trainset)

In [34]:
# 进行预测
predictions = algo.test(testset)

In [35]:
# 评估模型
rmse = accuracy.rmse(predictions) # 这行会打印并返回RMSE
mae = accuracy.mae(predictions) # 这行会打印并返回MAE

RMSE: 3.7810
MAE:  3.5570


In [73]:
# 推荐函数
def get_top_n_recommendations(User_ID, n=9):
    # 获取用户未评分的电影
    user_rated_books = ratings[ratings['User-ID'] == User_ID]['ISBN']
    all_books = set(ratings['ISBN'].unique())
    unrated_books = all_books - set(user_rated_books)
    # 预测用户对未评分电影的评分
    predictions = [algo.predict(User_ID, book_id) for book_id in unrated_books]
    # 按评分从高到低排序并返回前n个推荐结果
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    top_n_book_ids = [pred.iid for pred in top_n]
    top_n_estimates = [pred.est for pred in top_n]
    return books_df[jokes_df['ISBN'].isin(top_n_book_ids)][['ISBN', 'Book-Title']], top_n_estimates

In [80]:
import pandas as pd  
  
# 我们创建一个DataFrame  
data = {  
    'Book-Title': ['Book1', 'Book2', 'Book3', 'Book4', 'Book5', 'Book6', 'Book7', 'Book8'],  
    }  
  
# 创建一个DataFrame  
df = pd.DataFrame(data)  
  
# 现在df已经被定义，我们可以安全地获取其索引的长度  
index_length = len(df.index)  
print(index_length)  # 应该输出8  
  
# 有一个top_n_estimates列表，长度可能不对  
top_n_estimates = [1, 2, 3, 4, 5, 6, 7, 9, 10]  # 这里有9个值，而DataFrame有8行  
  
# 截断top_n_estimates到正确的长度  
top_n_estimates = top_n_estimates[:index_length]  
  
# 将截断后的列表赋值给DataFrame的一个新列  
df['predicted_rating'] = top_n_estimates  
  
# 现在df包含一个名为'predicted_rating'的新列  
print(df)

8
  Book-Title  predicted_rating
0      Book1                 1
1      Book2                 2
2      Book3                 3
3      Book4                 4
4      Book5                 5
5      Book6                 6
6      Book7                 7
7      Book8                 9


In [81]:
# 假设df是你的DataFrame，index_length是DataFrame的索引长度  
index_length = len(df.index)  
top_n_estimates = top_n_estimates[:index_length]  # 截断top_n_estimates到正确的长度  
df['predicted_rating'] = top_n_estimates


In [82]:
new_user_id = ratings['User-ID'].max() + 1
new_user_ratings = pd.DataFrame({
        'User-ID': [new_user_id,new_user_id,new_user_id],
        'ISBN': [107,149,34],
        'Book-Rating': [10,9,2],
    })

    # 将新用户的评分添加到训练集
new_ratings = pd.concat([ratings,new_user_ratings],ignore_index=True)
new_train_data = Dataset.load_from_df(new_ratings[['User-ID','ISBN','Book-Rating']],reader)
new_trainset = new_train_data.build_full_trainset()

    # 重新训练模型
algo.fit(new_trainset)

    # 为新用户生成推荐
recommended_books_new_user, top_n_estimates = get_top_n_recommendations(new_user_id)
recommended_books_new_user['predicted_rating'] = top_n_estimates

print(f"为新用户 {new_user_id} 推荐的前10个笑话: ")
print(recommended_books_new_user[['Book-Title', 'predicted_rating']])

为新用户 278855 推荐的前10个笑话: 
                                              Book-Title  predicted_rating
101    Rich Dad, Poor Dad: What the Rich Teach Their ...          8.101867
748                                    The Da Vinci Code          7.293884
1101        Where the Sidewalk Ends : Poems and Drawings          7.198364
1595                          The Blue Nowhere : A Novel          7.183948
3028                                                Free          7.153900
4697   Stranger in a Strange Land (Remembering Tomorrow)          7.126729
9785            The Vagina Monologues: The V-Day Edition          7.030364
11220  Marching Through Culpeper : A Novel of Culpepe...          7.009743
16190                                         Falling Up          6.938264


In [42]:
print(ratings.columns)

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [75]:
print(len(recommended_books_new_user.index))  # 应该输出8

8


In [76]:
print(len(top_n_estimates))  # 应该输出8

9
